In [2]:
from splinter import Browser
from bs4 import BeautifulSoup
import re
import time
import requests
import time as t
from commonregex import CommonRegex
from streetaddress import StreetAddressFormatter, StreetAddressParser
import pandas as pd

In [3]:
!which chromedriver

/opt/homebrew/bin/chromedriver


In [4]:
executable_path = {'executable_path': '/opt/homebrew/bin/chromedriver'}
browser = Browser('chrome', **executable_path, headless=False)

In [5]:
url = "https://www.golflink.com/golf-courses"
State = "CA" 

City = "La Quinta"

st_city_url = url + "/" + State.lower() + "/" + City.replace(" ","-").lower() + "/"
st_city_url

'https://www.golflink.com/golf-courses/ca/la-quinta/'

In [6]:
browser.visit(st_city_url)

In [7]:
html = browser.html
soup = BeautifulSoup(html, 'html.parser')

In [8]:
course_dict = {}
dict_list = []

courses = soup.find_all("div", class_="resultContainer")
for course in courses:
    course_dict = {}
    name = course.find("a", class_="fly")
    address_blurb = course.find("div", class_ = "textcontainer")
    text_add_blur = address_blurb.text
    clean_add_blurb = " ".join(text_add_blur.split())
    course_dict["Name"] = name.text.split(',')[0]
    course_dict["Course Name"] = name.text.split(',')[1]
    course_dict["Holes"] = re.search(r'.*holes', clean_add_blurb)[0]
    try:
        pub_priv = re.findall('(Private|Public|Municipal)', clean_add_blurb)[0]
        course_dict["Public/Private"] = pub_priv
    except:
        course_dict["Public/Private"] = "N/A"
    course_dict["Phone"] = re.findall("\(\w{3}\) \w{3}-\w{4}",clean_add_blurb)[0]

    phone = re.findall("\(\w{3}\) \w{3}-\w{4}",clean_add_blurb)
    add = clean_add_blurb.rpartition(phone[0])[0]
    course_dict["Address"] = (re.split('Write a Review Check Tee Times |Write a Review ', add)[-1])
    course_dict["Blurb"] = clean_add_blurb
    dict_list.append(course_dict)


In [9]:
dict_list

[{'Name': 'Tradition Golf Club',
  'Course Name': ' Tradition Course',
  'Holes': '18 holes',
  'Public/Private': 'Private',
  'Phone': '(760) 564-1067',
  'Address': '78505 Old Avenue 52 La Quinta, CA 92253 ',
  'Blurb': '18 holes over 6,915 yards with a par of 72 (Private) Write a Review 78505 Old Avenue 52 La Quinta, CA 92253 (760) 564-1067 0 miles from the center of La Quinta. Write a Review'},
 {'Name': 'Tradition Golf Club',
  'Course Name': ' Par-3 Course',
  'Holes': '9 holes',
  'Public/Private': 'Private',
  'Phone': '(760) 564-1067',
  'Address': '78505 Old Avenue 52 La Quinta, CA 92253 ',
  'Blurb': '9 holes (Private) Write a Review 78505 Old Avenue 52 La Quinta, CA 92253 (760) 564-1067 0 miles from the center of La Quinta. Write a Review'},
 {'Name': 'PGA West Mountain & Dunes Golf Courses',
  'Course Name': ' Pete Dye Mountain Course',
  'Holes': '18 holes',
  'Public/Private': 'Public',
  'Phone': '(760) 564-7610',
  'Address': '50200 Avenida Vista Bonita La Quinta, CA 9

In [10]:
df = pd.DataFrame.from_dict(dict_list)
df

,Name,Course Name,Holes,Public/Private,Phone,Address,Blurb
0,Tradition Golf Club,Tradition Course,18 holes,Private,(760) 564-1067,"78505 Old Avenue 52 La Quinta, CA 92253","18 holes over 6,915 yards with a par of 72 (Pr..."
1,Tradition Golf Club,Par-3 Course,9 holes,Private,(760) 564-1067,"78505 Old Avenue 52 La Quinta, CA 92253",9 holes (Private) Write a Review 78505 Old Ave...
2,PGA West Mountain & Dunes Golf Courses,Pete Dye Mountain Course,18 holes,Public,(760) 564-7610,"50200 Avenida Vista Bonita La Quinta, CA 92253...","18 holes over 6,756 yards with a par of 72 (Pu..."
3,PGA West Mountain & Dunes Golf Courses,Pete Dye Dunes Course,18 holes,Public,(760) 564-7610,"50200 Avenida Vista Bonita La Quinta, CA 92253...","18 holes over 6,074 yards with a par of 72 (Pu..."
4,La Quinta Country Club,La Quinta Course,18 holes,Private,(760) 564-4151,"77750 Avenue 50 La Quinta, CA 92253-2204","18 holes over 7,060 yards with a par of 72 (Pr..."
...,...,...,...,...,...,...,...
95,Marriott's Shadow Ridge Golf Club,Marriott Shadow Ridge Course,18 holes,Public,(760) 674-2700,"9002 Shadow Ridge Rd Palm Desert, CA 92211-2057","18 holes over 6,932 yards with a par of 71 (Pu..."
96,Rancho Mirage Country Club,Rancho Mirage Course,18 holes,Public,(760) 324-4711,"38500 Bob Hope Dr Rancho Mirage, CA 92270-3259","18 holes over 6,111 yards with a par of 70 (Pu..."
97,Desert Island Country Club,The S Course,18 holes,Private,(760) 328-0841,"71777 Frank Sinatra Dr Rancho Mirage, CA 92270...","18 holes over 6,715 yards with a par of 72 (Pr..."
98,Thunderbird Country Club,Thunderbird Course,18 holes,Private,(760) 328-2161,"70737 Country Club Dr Rancho Mirage, CA 92270-...","18 holes over 6,460 yards with a par of 71 (Pr..."


In [11]:
df.to_csv("Data_Scraped/La-Quinta_courses.csv", index=False)